# TCGA Methylation Data Importation
**Local Version**: 1
**Source Version**: NA

This notebook will import raw TCGA methylation data through the [CGDS](http://www.cbioportal.org/cgds_r.jsp) portal.

Note that there are two types of methylation data available for nearly all TCGA studies -- data from the HM27 platform and data from the HM450 (more precisely, "Illimunia Infinium HumanMethylation450 BeadChip Kit").  The latter is newer and larger so data for it will be imported instead.

In [3]:
%run -m ipy_startup
%run -m ipy_logging false
%matplotlib inline
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import data_type as dtyp
from mgds.data_aggregation import api
from mgds.data_aggregation.import_lib import cgds
from mgds.data_aggregation.import_lib import tcga
from py_utils.collection_utils import subset

In [5]:
tables = tcga.import_genetic_profile_data(
    profile_fmt=tcga.PROF_FMT_METHYLATION,
    data_type=dtyp.GENE_METHYLATION,
    gene_ids=api.get_hugo_gene_ids(),
    cohorts=['brca']
)

INFO:mgds.data_aggregation.import_lib.tcga:Importing data for study "brca_tcga" (3 of 32), cohort "brca", case list "brca_tcga_all", profile "brca_tcga_methylation_hm450", table "brca-gene-methylation"
2016-12-19 08:37:44,307:INFO:mgds.data_aggregation.import_lib.tcga: Importing data for study "brca_tcga" (3 of 32), cohort "brca", case list "brca_tcga_all", profile "brca_tcga_methylation_hm450", table "brca-gene-methylation"
INFO:mgds.data_aggregation.import_lib.cgds:Processing batch 1 of 789
2016-12-19 08:37:44,308:INFO:mgds.data_aggregation.import_lib.cgds: Processing batch 1 of 789
INFO:mgds.data_aggregation.import_lib.cgds:Processing batch 79 of 789
2016-12-19 08:39:31,002:INFO:mgds.data_aggregation.import_lib.cgds: Processing batch 79 of 789
INFO:mgds.data_aggregation.import_lib.cgds:Processing batch 157 of 789
2016-12-19 08:41:17,428:INFO:mgds.data_aggregation.import_lib.cgds: Processing batch 157 of 789
INFO:mgds.data_aggregation.import_lib.cgds:Processing batch 235 of 789
2016-

In [7]:
d = cgds.melt_raw_data(d)
d.info()

NameError: name 'd' is not defined

In [5]:
d_agg, d_dist = cgds.aggregate(d)
d_agg.head()

,CELL_LINE_ID,GENE_ID:HGNC,GENE_ID:CGDS,VALUE_MEAN,VALUE_STD,VALUE_CT
0,TCGA-A1-A0SB-01,A1BG,1,0.073395,0.0,1
1,TCGA-A1-A0SB-01,A1BG-AS1,503538,0.724501,0.0,1
2,TCGA-A1-A0SB-01,A2M,2,0.891226,0.0,1
3,TCGA-A1-A0SB-01,A2ML1,144568,0.478400,0.0,1
4,TCGA-A1-A0SB-01,A4GALT,53947,0.513391,0.0,1


In [6]:
d_dist

1    8785063
2      11612
Name: Number of Replicates, dtype: int64

In [7]:
d_agg.describe()

,GENE_ID:CGDS,VALUE_MEAN,VALUE_STD,VALUE_CT
count,8.796675e+06,8.796675e+06,8796675.0,8.796675e+06
mean,5.930480e+05,4.078580e-01,0.0,1.001320e+00
std,7.169097e+06,3.489149e-01,0.0,3.630843e-02
min,1.000000e+00,3.360104e-03,0.0,1.000000e+00
25%,6.774000e+03,4.721479e-02,0.0,1.000000e+00
50%,2.734700e+04,3.495343e-01,0.0,1.000000e+00
75%,8.466200e+04,7.720465e-01,0.0,1.000000e+00
max,1.005291e+08,9.962133e-01,0.0,2.000000e+00


In [8]:
assert np.all(pd.notnull(d_agg))
db.save(d_agg, src.TCGA_BREAST_v1, db.IMPORT, 'gene-methylation')

'/Users/eczech/data/research/mgds/import/tcga-breast_v1_gene-methylation.pkl'